# Encoder-Decoder Architecture

In general seq2seq problems 
like machine translation,
inputs and outputs are of varying lengths
that are unaligned. 
The standard approach to handling this sort of data
is to design an *encoder-decoder* architecture
consisting of two major components:
an *encoder* that takes a variable-length sequence as input,
and a *decoder* that acts as a conditional language model,
taking in the encoded input 
and the leftwards context of the target sequence 
and predicting the subsequent token in the target sequence. 

Let's take machine translation from English to French as an example.
Given an input sequence in English:
"They", "are", "watching", ".",
this encoder-decoder architecture
first encodes the variable-length input into a state,
then decodes the state 
to generate the translated sequence,
token by token, as output:
"Ils", "regardent", ".".
Since the encoder-decoder architecture
forms the basis of different seq2seq models
in subsequent sections,
this section will convert this architecture
into an interface that will be implemented later.

## Encoder

In the encoder interface,
we just specify that
the encoder takes variable-length sequences as input `X`.
The implementation will be provided 
by any model that inherits this base `Encoder` class.


In [1]:
from torch import nn

class Encoder(nn.Module):
    """The base encoder interface for the encoder-decoder architecture."""
    def __init__(self, **kwargs):
        super(Encoder, self).__init__(**kwargs)
    
    # Later there can be additional arguments (e.g., length excluding padding)
    def forward(self, X, *args):
        raise NotImplementedError

## Decoder

In the following decoder interface,
we add an additional `init_state` function
to convert the encoder output (`enc_all_outputs`)
into the encoded state.
Note that this step
may require extra inputs,
such as the valid length of the input,
which was explained
in Machine Translation.
To generate a variable-length sequence token by token,
every time the decoder may map an input 
(e.g., the generated token at the previous time step)
and the encoded state 
into an output token at the current time step.


In [2]:
class Decoder(nn.Module):  
    """The base decoder interface for the encoder-decoder architecture."""
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)

    # Later there can be additional arguments (e.g., length excluding padding)
    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

## Putting the Encoder and Decoder Together

In the forward propagation,
the output of the encoder
is used to produce the encoded state,
and this state will be further used
by the decoder as one of its input.


In [4]:
class EncoderDecoder(nn.Module):
    """The base class for the encoder-decoder architecture."""
    def __init__(self, encoder, decoder, **kwargs):
        super(EncoderDecoder, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, enc_X, dec_X, *args):
        enc_outputs = self.encoder(enc_X, *args)
        dec_state = self.decoder.init_state(enc_outputs, *args)
        # return decoder output only
        return self.decoder(dec_X, dec_state)

In the next section, 
we will see how to apply RNNs to design 
seq2seq models based on 
this encoder-decoder architecture.


## Summary

Encoder-decoder architectures
can handle inputs and outputs 
that both consist of variable-length sequences
and thus are suitable for seq2seq problems 
such as machine translation.
The encoder takes a variable-length sequence as input 
and transforms it into a state with a fixed shape.
The decoder maps the encoded state of a fixed shape
to a variable-length sequence.